In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
import torch_geometric
import pickle
import optuna
from torch_geometric.nn import GCNConv

### data

In [2]:
# model structure
class GCN(torch.nn.Module):
    def __init__(self, size1, size2, size3, size4):
        super(GCN, self).__init__()

        self.conv1 = GCNConv(1969, size1)
        self.conv2 = GCNConv(size1, size2)
        self.conv3 = GCNConv(size2, size3)
        self.fc1 = torch.nn.Linear(size3, size4)
        self.fc2 = torch.nn.Linear(size4, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.conv3(x, edge_index)
        x1 = torch.relu(x)
        x = self.fc1(x1)
        x = torch.relu(x)
        x = self.fc2(x)       
        return x, x1

def train(model, data, optimizer, mask):
    model.train()
    optimizer.zero_grad()
    out, _ = model(data)
    loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    loss.backward()
    optimizer.step()
    return loss.item()

def evaluate(model, data, mask):
    model.eval()
    with torch.no_grad():
        out, _ = model(data)
        loss = F.mse_loss(out[mask].view(-1), data.y[mask])
    return loss.item()

def objective(trial: optuna.Trial):   
    size1 = trial.suggest_int("size1", 32, 1024)
    size2 = trial.suggest_int("size2", 4, 512)
    size3 = trial.suggest_int("size3", 4, 512)
    size4 = trial.suggest_int("size4", 4, 512)
    num_epochs = trial.suggest_int("num_epochs", 1, 200)
    lr = trial.suggest_float("lr", 1e-5, 1e-2)

    model = GCN(size1, size2, size3, size4).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # find best hyperparameter by training on the training set and get loss on the validating set
    for epoch in range(1, num_epochs + 1):
        train_loss = train(model, data, optimizer, data.train_mask)
        val_loss = evaluate(model, data, data.valid_mask)
        trial.report(val_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    return val_loss

In [3]:
for rs in range(10):
    
    # read data
    X_train = pd.read_csv("../../result/input_perturb_phyloP/%d/X_train_stratified" % rs, sep="\t", index_col=0).values
    X_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/X_valid_stratified" % rs, sep="\t", index_col=0).values
    X_test = pd.read_csv("../../result/input_perturb_phyloP/%d/X_test_stratified" % rs, sep="\t", index_col=0).values
    Y_train = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_train_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_valid = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_valid_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).values.reshape(-1)
    Y_test_gene = pd.read_csv("../../result/input_perturb_phyloP/%d/Y_test_stratified" % rs, sep="\t", index_col=0).index

    # concat all subsets
    X = np.concatenate([X_train, X_valid, X_test])
    Y = np.concatenate([Y_train, Y_valid, Y_test])
    train_mask = np.concatenate([[True] * len(X_train), [False] * len(X_valid), [False] * len(X_test)])
    valid_mask = np.concatenate([[False] * len(X_train), [True] * len(X_valid), [False] * len(X_test)])
    test_mask = np.concatenate([[False] * len(X_train), [False] * len(X_valid), [True] * len(X_test)])
    
    # mask of whether a test node is in the DAGMA graph
    dag = pd.read_csv("../../result/network_perturb_phyloP/DAGMA_thresholdAdaptive.tsv", sep="\t", header=None)
    id2genes = pd.read_csv("../../result/network_perturb_phyloP/valid_genes", sep="\t").set_index("ID")['genes'].to_dict()
    dag[0] = dag[0].map(id2genes)
    dag[1] = dag[1].map(id2genes)
    dag_genes = list(set.union(set(dag[0]), set(dag[1])))
    test_inDAG = Y_test_gene.isin(dag_genes)

    for network_name in ["DAGMA_thresholdAdaptive", "STRING", "BIOGRID", "Combine", "ChIP_hTFtarget", "ChIP_TIP", "ChIP_TIP_K562", "CoExpr_ENCODE_K562_0.75", "CoExpr_GTEx_WholeBlood_0.75", "CoExpr_perturb_0.5", "Random_ER", "Random_SF", "NOTEARS_thresholdAdaptive"]:

        # read network
        edge_index = torch.tensor(pd.read_csv("../../result/network_perturb_phyloP/%s.tsv" % network_name, sep="\t", header=None).values.T)

        # convert to data object
        data = torch_geometric.data.Data(x=torch.tensor(X).float(), edge_index=edge_index, y=torch.tensor(Y).float())
        data.train_mask = torch.tensor(train_mask)
        data.valid_mask = torch.tensor(valid_mask)
        data.test_mask = torch.tensor(test_mask)
        data.test_net_mask = np.concatenate([[False] * len(Y_train), [False] * len(Y_valid), test_inDAG])

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        data = data.to(device)


        # repeat the whole process multiple times for mean and std of the loss
        test_loss_reps = []
        test_net_loss_reps = []

        os.makedirs("../../result/model_perturb_phyloP_deep3/%s/%d/" % (network_name, rs), exist_ok=True)

        for rep in range(10):
            # tune hyperparameters
            study = optuna.create_study(direction="minimize")
            study.optimize(objective, n_trials=20, timeout=600)
            best_params = study.best_params

            # final model with best hyperparameters and use train and valid data to train
            final_model = GCN(best_params["size1"], best_params["size2"], best_params["size3"], best_params["size4"]).to(device)
            optimizer = torch.optim.Adam(final_model.parameters(), lr=best_params["lr"])

            # train final model
            for epoch in range(1, best_params["num_epochs"] + 1):
                train_loss = train(final_model, data, optimizer, data.train_mask|data.valid_mask)

            # prediction from final model
            final_model.eval()
            with torch.no_grad():
                out, X1 = final_model(data)
                # loss on the testing set
                loss1 = F.mse_loss(out[data.test_mask].view(-1), data.y[data.test_mask])
                test_loss_reps.append(loss1.item())
                # loss on the testing set, but only the nodes in the DAGMA
                loss2 = F.mse_loss(out[data.test_net_mask].view(-1), data.y[data.test_net_mask])
                test_net_loss_reps.append(loss2.item())

            # save model, hyperparameters and results
            prefix = "../../result/model_perturb_phyloP_deep3/%s/%d/model%d" % (network_name, rs, rep)
            torch.save(final_model.state_dict(), prefix+".model")
            with open(prefix+".para", "wb") as f:
                pickle.dump([best_params, X1.cpu().numpy(), out.cpu().numpy(), loss1.item(), loss2.item()], f)

[I 2023-12-05 13:03:16,324] A new study created in memory with name: no-name-ef95e01a-0a4c-4a19-b110-9cbba9a6f8cf
[I 2023-12-05 13:03:21,805] Trial 0 finished with value: 0.2501920759677887 and parameters: {'size1': 226, 'size2': 25, 'size3': 343, 'size4': 117, 'num_epochs': 9, 'lr': 0.009813336129569555}. Best is trial 0 with value: 0.2501920759677887.
[I 2023-12-05 13:03:21,843] Trial 1 finished with value: 448.4083251953125 and parameters: {'size1': 900, 'size2': 470, 'size3': 340, 'size4': 103, 'num_epochs': 1, 'lr': 0.0069000676920293914}. Best is trial 0 with value: 0.2501920759677887.
[I 2023-12-05 13:03:22,267] Trial 2 finished with value: 0.1466633379459381 and parameters: {'size1': 340, 'size2': 15, 'size3': 402, 'size4': 107, 'num_epochs': 72, 'lr': 0.004221480384301477}. Best is trial 2 with value: 0.1466633379459381.
[I 2023-12-05 13:03:22,473] Trial 3 finished with value: 0.14685048162937164 and parameters: {'size1': 950, 'size2': 41, 'size3': 427, 'size4': 264, 'num_epoc

[I 2023-12-05 13:03:38,182] Trial 3 finished with value: 0.15988397598266602 and parameters: {'size1': 834, 'size2': 389, 'size3': 492, 'size4': 333, 'num_epochs': 160, 'lr': 0.005792266475277893}. Best is trial 2 with value: 0.15554755926132202.
[I 2023-12-05 13:03:38,229] Trial 4 finished with value: 0.18948298692703247 and parameters: {'size1': 824, 'size2': 464, 'size3': 328, 'size4': 454, 'num_epochs': 3, 'lr': 0.0017589226434881434}. Best is trial 2 with value: 0.15554755926132202.
[I 2023-12-05 13:03:38,244] Trial 5 pruned. 
[I 2023-12-05 13:03:38,266] Trial 6 pruned. 
[I 2023-12-05 13:03:38,304] Trial 7 pruned. 
[I 2023-12-05 13:03:38,344] Trial 8 pruned. 
[I 2023-12-05 13:03:38,885] Trial 9 finished with value: 0.14934709668159485 and parameters: {'size1': 355, 'size2': 115, 'size3': 118, 'size4': 446, 'num_epochs': 89, 'lr': 0.006516305338694782}. Best is trial 9 with value: 0.14934709668159485.
[I 2023-12-05 13:03:38,927] Trial 10 pruned. 
[I 2023-12-05 13:03:39,332] Trial 1

[I 2023-12-05 13:03:53,805] Trial 6 pruned. 
[I 2023-12-05 13:03:53,818] Trial 7 pruned. 
[I 2023-12-05 13:03:53,844] Trial 8 pruned. 
[I 2023-12-05 13:03:53,860] Trial 9 pruned. 
[I 2023-12-05 13:03:53,920] Trial 10 pruned. 
[I 2023-12-05 13:03:53,975] Trial 11 pruned. 
[I 2023-12-05 13:03:54,031] Trial 12 pruned. 
[I 2023-12-05 13:03:54,071] Trial 13 pruned. 
[I 2023-12-05 13:03:54,104] Trial 14 pruned. 
[I 2023-12-05 13:03:54,152] Trial 15 pruned. 
[I 2023-12-05 13:03:54,191] Trial 16 pruned. 
[I 2023-12-05 13:03:54,234] Trial 17 pruned. 
[I 2023-12-05 13:03:54,291] Trial 18 pruned. 
[I 2023-12-05 13:03:54,340] Trial 19 pruned. 
[I 2023-12-05 13:03:54,973] A new study created in memory with name: no-name-aaf843ce-6dfe-4852-b3ab-b42195b82011
[I 2023-12-05 13:03:55,468] Trial 0 finished with value: 0.15767942368984222 and parameters: {'size1': 924, 'size2': 196, 'size3': 301, 'size4': 237, 'num_epochs': 72, 'lr': 0.006900815609988349}. Best is trial 0 with value: 0.15767942368984222.


[I 2023-12-05 13:04:18,078] Trial 4 finished with value: 0.16964127123355865 and parameters: {'size1': 163, 'size2': 128, 'size3': 396, 'size4': 262, 'num_epochs': 196, 'lr': 0.0003509458965738095}. Best is trial 3 with value: 0.15960131585597992.
[I 2023-12-05 13:04:18,122] Trial 5 pruned. 
[I 2023-12-05 13:04:18,151] Trial 6 pruned. 
[I 2023-12-05 13:04:18,194] Trial 7 pruned. 
[I 2023-12-05 13:04:18,234] Trial 8 pruned. 
[I 2023-12-05 13:04:18,256] Trial 9 pruned. 
[I 2023-12-05 13:04:18,305] Trial 10 pruned. 
[I 2023-12-05 13:04:18,357] Trial 11 pruned. 
[I 2023-12-05 13:04:18,400] Trial 12 pruned. 
[I 2023-12-05 13:04:18,463] Trial 13 pruned. 
[I 2023-12-05 13:04:18,542] Trial 14 pruned. 
[I 2023-12-05 13:04:18,587] Trial 15 pruned. 
[I 2023-12-05 13:04:18,638] Trial 16 pruned. 
[I 2023-12-05 13:04:18,700] Trial 17 pruned. 
[I 2023-12-05 13:04:19,211] Trial 18 finished with value: 0.15656016767024994 and parameters: {'size1': 150, 'size2': 213, 'size3': 396, 'size4': 12, 'num_epoc

[I 2023-12-05 13:04:57,262] Trial 6 pruned. 
[I 2023-12-05 13:04:57,293] Trial 7 pruned. 
[I 2023-12-05 13:04:57,339] Trial 8 pruned. 
[I 2023-12-05 13:04:57,384] Trial 9 pruned. 
[I 2023-12-05 13:04:57,458] Trial 10 finished with value: 0.1599704623222351 and parameters: {'size1': 101, 'size2': 304, 'size3': 338, 'size4': 507, 'num_epochs': 4, 'lr': 0.003339483337355677}. Best is trial 2 with value: 0.15491808950901031.
[I 2023-12-05 13:04:57,501] Trial 11 pruned. 
[I 2023-12-05 13:04:57,553] Trial 12 pruned. 
[I 2023-12-05 13:04:57,596] Trial 13 pruned. 
[I 2023-12-05 13:04:57,647] Trial 14 pruned. 
[I 2023-12-05 13:04:57,689] Trial 15 pruned. 
[I 2023-12-05 13:04:57,745] Trial 16 pruned. 
[I 2023-12-05 13:04:57,796] Trial 17 pruned. 
[I 2023-12-05 13:04:57,838] Trial 18 pruned. 
[I 2023-12-05 13:04:57,876] Trial 19 pruned. 
[I 2023-12-05 13:04:58,632] A new study created in memory with name: no-name-fca0626a-40e5-42c0-8385-cd4980934ce6
[I 2023-12-05 13:04:59,020] Trial 0 finished wi

...

[I 2023-12-05 18:13:10,674] Trial 16 pruned. 
[I 2023-12-05 18:13:10,728] Trial 17 pruned. 
[I 2023-12-05 18:13:10,765] Trial 18 pruned. 
[I 2023-12-05 18:13:10,811] Trial 19 pruned. 
[I 2023-12-05 18:13:11,565] A new study created in memory with name: no-name-d4914ad7-13f3-43d0-ba78-08fb6328e91b
[I 2023-12-05 18:13:12,476] Trial 0 finished with value: 0.1733962893486023 and parameters: {'size1': 713, 'size2': 401, 'size3': 46, 'size4': 375, 'num_epochs': 111, 'lr': 0.005006394226225869}. Best is trial 0 with value: 0.1733962893486023.
[I 2023-12-05 18:13:12,613] Trial 1 finished with value: 0.24131512641906738 and parameters: {'size1': 813, 'size2': 294, 'size3': 178, 'size4': 29, 'num_epochs': 14, 'lr': 0.00316491345089309}. Best is trial 0 with value: 0.1733962893486023.
[I 2023-12-05 18:13:13,816] Trial 2 finished with value: 0.17262311279773712 and parameters: {'size1': 921, 'size2': 350, 'size3': 293, 'size4': 182, 'num_epochs': 135, 'lr': 0.009946663927753197}. Best is trial 2 w

[I 2023-12-05 18:13:37,082] Trial 2 finished with value: 0.29409152269363403 and parameters: {'size1': 785, 'size2': 322, 'size3': 168, 'size4': 146, 'num_epochs': 190, 'lr': 0.0024684382954471007}. Best is trial 0 with value: 0.16940875351428986.
[I 2023-12-05 18:13:37,277] Trial 3 finished with value: 0.18088670074939728 and parameters: {'size1': 381, 'size2': 118, 'size3': 273, 'size4': 189, 'num_epochs': 29, 'lr': 0.003365102112594792}. Best is trial 0 with value: 0.16940875351428986.
[I 2023-12-05 18:13:38,674] Trial 4 finished with value: 0.17653828859329224 and parameters: {'size1': 900, 'size2': 295, 'size3': 296, 'size4': 30, 'num_epochs': 162, 'lr': 0.004030311023660414}. Best is trial 0 with value: 0.16940875351428986.
[I 2023-12-05 18:13:38,696] Trial 5 pruned. 
[I 2023-12-05 18:13:38,816] Trial 6 pruned. 
[I 2023-12-05 18:13:38,853] Trial 7 pruned. 
[I 2023-12-05 18:13:39,334] Trial 8 finished with value: 0.19881126284599304 and parameters: {'size1': 231, 'size2': 478, 'si

[I 2023-12-05 18:13:56,232] Trial 5 finished with value: 0.1561139076948166 and parameters: {'size1': 917, 'size2': 486, 'size3': 345, 'size4': 372, 'num_epochs': 48, 'lr': 0.0024750678406589236}. Best is trial 4 with value: 0.13645108044147491.
[I 2023-12-05 18:13:56,477] Trial 6 finished with value: 0.1444549709558487 and parameters: {'size1': 612, 'size2': 148, 'size3': 179, 'size4': 436, 'num_epochs': 37, 'lr': 0.0018945124796981318}. Best is trial 4 with value: 0.13645108044147491.
[I 2023-12-05 18:13:56,501] Trial 7 pruned. 
[I 2023-12-05 18:13:56,521] Trial 8 pruned. 
[I 2023-12-05 18:13:56,544] Trial 9 pruned. 
[I 2023-12-05 18:13:56,584] Trial 10 pruned. 
[I 2023-12-05 18:13:56,637] Trial 11 pruned. 
[I 2023-12-05 18:13:56,690] Trial 12 pruned. 
[I 2023-12-05 18:13:56,741] Trial 13 pruned. 
[I 2023-12-05 18:13:56,798] Trial 14 pruned. 
[I 2023-12-05 18:13:56,843] Trial 15 pruned. 
[I 2023-12-05 18:13:56,902] Trial 16 pruned. 
[I 2023-12-05 18:13:56,963] Trial 17 pruned. 
[I 20

[I 2023-12-05 18:14:14,611] Trial 8 pruned. 
[I 2023-12-05 18:14:14,643] Trial 9 pruned. 
[I 2023-12-05 18:14:14,691] Trial 10 pruned. 
[I 2023-12-05 18:14:15,474] Trial 11 finished with value: 0.14550510048866272 and parameters: {'size1': 76, 'size2': 137, 'size3': 24, 'size4': 143, 'num_epochs': 129, 'lr': 0.002367737568932948}. Best is trial 3 with value: 0.13886462152004242.
[I 2023-12-05 18:14:15,512] Trial 12 pruned. 
[I 2023-12-05 18:14:15,594] Trial 13 pruned. 
[I 2023-12-05 18:14:16,490] Trial 14 finished with value: 0.12990622222423553 and parameters: {'size1': 277, 'size2': 6, 'size3': 99, 'size4': 329, 'num_epochs': 148, 'lr': 0.0018912863542515286}. Best is trial 14 with value: 0.12990622222423553.
[I 2023-12-05 18:14:16,540] Trial 15 pruned. 
[I 2023-12-05 18:14:17,518] Trial 16 finished with value: 0.1300365924835205 and parameters: {'size1': 390, 'size2': 327, 'size3': 94, 'size4': 409, 'num_epochs': 158, 'lr': 0.0022087146434759926}. Best is trial 14 with value: 0.1299

[I 2023-12-05 18:14:35,994] Trial 13 pruned. 
[I 2023-12-05 18:14:36,055] Trial 14 pruned. 
[I 2023-12-05 18:14:36,607] Trial 15 pruned. 
[I 2023-12-05 18:14:37,204] Trial 16 finished with value: 0.13900598883628845 and parameters: {'size1': 594, 'size2': 148, 'size3': 378, 'size4': 455, 'num_epochs': 90, 'lr': 0.0015265760204925638}. Best is trial 4 with value: 0.13455304503440857.
[I 2023-12-05 18:14:37,249] Trial 17 pruned. 
[I 2023-12-05 18:14:37,308] Trial 18 pruned. 
[I 2023-12-05 18:14:37,525] Trial 19 pruned. 
[I 2023-12-05 18:14:38,102] A new study created in memory with name: no-name-59602ab9-0cb9-4c04-b345-ef0ee9c50c52
[I 2023-12-05 18:14:39,151] Trial 0 finished with value: 0.1744488626718521 and parameters: {'size1': 693, 'size2': 331, 'size3': 403, 'size4': 360, 'num_epochs': 165, 'lr': 0.007889694546267605}. Best is trial 0 with value: 0.1744488626718521.
[I 2023-12-05 18:14:39,249] Trial 1 finished with value: 0.16718025505542755 and parameters: {'size1': 908, 'size2': 